In [16]:
#pip install edgartools

In [17]:
# !pip3 install edgartools
# !pip3 install pandas
# !pip3 install polars
# !pip install tqdm
# !pip install html5lib
from tqdm import tqdm
import pandas as pd
import polars as pl
from edgar import set_identity
set_identity("jason.xu071498@gmail.com")
from edgar import *
import time
import pickle
import sys

[19:04:55] INFO     Identity of the Edgar REST client set to [jason.xu071498@gmail.com]                 ]8;id=961833;file:///Users/yanzhe.li/anaconda3/envs/sec/lib/python3.10/site-packages/edgar/core.py\core.py]8;;\:]8;id=447356;file:///Users/yanzhe.li/anaconda3/envs/sec/lib/python3.10/site-packages/edgar/core.py#153\153]8;;\

In [18]:
START_YEAR = 2022
END_YEAR = 2023

In [19]:
# Fetch filings for the specified year range
# filing_10k = Company("MCO").get_filings(form="10-K").latest(40)
# Filter for 10-K forms
# filing_10k = filings.filter(form=["10-K"])
filing_10k_by_year = {}
for year in range(START_YEAR, END_YEAR+1):
    # filing_10k = get_filings(year=year).filter(form=["10-K"])
    time.sleep(1)
    filing_10k = get_filings(year=year).filter(form=["10-K"]).latest(1)
    filing_10k_by_year[year] = filing_10k #?

In [20]:
def contains_trade_secret_column(df):
    # List of column names to search for
    search_columns = ["trade secret", "trade secrets", "net trade secret", "net trade secrets", "trade secrets (not subject to amortization)", "trade secrecy", "net trade secrecy"]

    # Filter rows where the first column is a string, then check if any value matches the search_columns list
    filtered_df = df[df.iloc[:, 0].apply(lambda x: isinstance(x, str))] #?
    result = None
    try:
        result = any(filtered_df.iloc[:, 0].str.lower().isin(search_columns))
    except:
        return None
    return result

In [21]:
def contains_trade_secret(content):
    # Check if 'trade secret' or 'trade secrecy' is in the content
    return 'trade secret' in content.lower() or 'trade secrecy' in content.lower()

In [23]:
def download_attachments(filing_10k_by_year, file_name, year):
    # Initialize an empty dictionary to store attachments.
    filing_attachments = {}  # key: attachment URL, value: file object of the html attachment

    # Loop through each filing in the provided list of 10-K filings.
    for _, filing in enumerate(filing_10k_by_year[year]):
        # Check if the current filing has any attachments.
        if filing.attachments:
            # Select the first attachment of the current filing.
            first_attachment = filing.attachments[0]
            print(first_attachment.url)
            # Download the attachment and store it in the dictionary with its URL as the key.
            filing_attachments[first_attachment.url] = first_attachment.download()
            time.sleep(1)

    # Open a file with the given file name in write-binary mode.
    with open(f'{file_name}_{year}.pkl', 'wb') as file:
        # Serialize and save the dictionary of attachments to the file.
        pickle.dump(filing_attachments, file)


def load_attachments(file_name, year):
    # Open the specified file in read-binary mode.
    with open(f'{file_name}_{year}.pkl', 'rb') as file:
        # Load and deserialize the data from the file.
        loaded_data = pickle.load(file)
        # Return the deserialized data.
        return loaded_data

FILE_NAME = "filing_attachments"

for i in tqdm(range(START_YEAR, END_YEAR+1)): # inclusive
    time.sleep(1)
    download_attachments(filing_10k_by_year, FILE_NAME, i) # ONLY CALL THIS FUNCTION WHEN YOU NEED TO RE-DOWNLOAD ATTACHMENTS

  0%|          | 0/1 [00:01<?, ?it/s]


TypeError: 'Filing' object is not iterable

In [ ]:
# Create an empty DataFrame with specified columns
results_df =  pd.DataFrame(columns=["company_name", "filing_date", "cik", "trade_secrets", "net_trade_secrets","indicator", "url", "accession_no"])

# Assuming filing_10k is a list of filings
for year in range(START_YEAR, END_YEAR+1):
    stored_attachments = load_attachments(FILE_NAME,year)
    for i, filing in enumerate(filing_10k_by_year[year]):
        filing_date = filing.filing_date
        # Note: write a script to scrape conformed_year_dict
        # conformed_year = conformed_year_dict.get(filing.accession_no)
        cik = filing.cik
        company_name = filing.company
        indicator = 0
        url = None
        accession_no = filing.accession_no

        if filing.attachments:
            list_df = None
            try:
                first_attachment_url = filing.attachments[0].url
                print(type(url))
                content = stored_attachments.get(first_attachment_url)
                list_df = pd.read_html(content)
                url = first_attachment_url

                if contains_trade_secret(content):
                    indicator = 1


            except Exception as e:
                print(f'Error happened while calling read_html() for {filing_date}, {company_name}: {str(e)}')
                list_df = []

            # Variable to store the DataFrame if found
            target_trade_secret_form = None

            # Iterate over each DataFrame in the list
            for form_df in list_df:
                # Check if the string 'trade secret' is present in the DataFrame (case-insensitive)
                if 'trade secret' in form_df.to_markdown().lower():
                # if contains_trade_secret_column(form_df):
                    # If found, assign this DataFrame to variable 'a' and break the loop
                    target_trade_secret_form = form_df
                    break

            # Check if a DataFrame containing 'trade secret' was found
            if target_trade_secret_form is not None:
                # Convert the pandas DataFrame 'target_k10_form' to a Polars DataFrame for further processing
                target_trade_secret_form = pl.from_pandas(target_trade_secret_form)
                trade_secrets: float = None
                net_trade_secrets: float = None
                # Iterate over each row in the Polars DataFrame
                for form_row in target_trade_secret_form.rows():
                    # Check if the first element of the row is not null or empty
                    if form_row[0]:
                        # Check if the string 'secret' is present in the first element (case-insensitive)
                        if 'secret' in form_row[0]:
                            # Iterate over each element in the current row
                            for item in form_row:
                                try:
                                    # Attempt to convert the element to a float and check if it's greater than 0
                                    if float(item) > 0:
                                        if form_row[0].lower() == "net trade secret" or form_row[0].lower() == "net trade secrets" or  form_row[0].lower() == "net trade secrecy":
                                            net_trade_secrets = float(item)
                                            break
                                        # elif form_row[0].lower() == "trade secret" or form_row[0].lower() == "trade secrets" or form_row[0].lower() == "trade secrets (not subject to amortization)":
                                        elif  "trade secret" in form_row[0].lower() or "trade secrecy"  in form_row[0].lower():
                                            try:
                                                trade_secrets = float(item)
                                            except Exception as e:
                                                print(f"Edge case: {filing_date} {cik} {company_name}") # debug only
                                                break
                                            break
                                        else:
                                            print(form_row[0])
                                            print("Unexpected case") # debug only
                                except ValueError:
                                    # If an error occurs (e.g., when the element cannot be converted to a float), continue to the next element
                                    continue
                                except TypeError:
                                    continue

                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": trade_secrets,
                    "net_trade_secrets": net_trade_secrets,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
            else:
                new_row = pd.DataFrame([{
                    "company_name": company_name,
                    "filing_date": filing_date,
                    "cik": cik,
                    "trade_secrets": None,
                    "net_trade_secrets": None,
                    "indicator": indicator,
                    "url": url,
                    "accession_no": accession_no
                }])
                # Concatenate the new row to the results DataFrame
                results_df = pd.concat([results_df, new_row], ignore_index=True)
                print(f"No DataFrame contains 'trade secret' in filing {i}.")

# Display the final results DataFrame
print(results)


In [11]:
results_df

,company_name,filing_date,cik,trade_secrets,net_trade_secrets,indicator,url,accession_no
0,ACCENTIA BIOPHARMACEUTICALS INC,2006-12-29,1310094,NaN,None,1,https://www.sec.gov/Archives/edgar/data/131009...,0001193125-06-261727
1,ACTIVIDENTITY CORP,2006-12-29,1183941,NaN,None,1,https://www.sec.gov/Archives/edgar/data/118394...,0001193125-06-261251
2,ALLSTATES WORLDCARGO INC,2006-12-29,1072293,NaN,None,1,https://www.sec.gov/Archives/edgar/data/107229...,0001260415-06-000061
3,ALTIGEN COMMUNICATIONS INC,2006-12-29,1003607,NaN,None,1,https://www.sec.gov/Archives/edgar/data/100360...,0001104659-06-084411
4,AMCON DISTRIBUTING CO,2006-12-29,928465,NaN,None,0,https://www.sec.gov/Archives/edgar/data/928465...,0000928465-06-000077
...,...,...,...,...,...,...,...,...
175,JANEL CORP,2023-12-08,1133062,None,None,1,https://www.sec.gov/Archives/edgar/data/113306...,0001140361-23-057011
176,JOHNSON OUTDOORS INC,2023-12-08,788329,None,None,1,https://www.sec.gov/Archives/edgar/data/788329...,0001140361-23-056923
177,"LEE ENTERPRISES, Inc",2023-12-08,58361,None,None,0,https://www.sec.gov/Archives/edgar/data/58361/...,0001628280-23-041119
178,PENNANTPARK INVESTMENT CORP,2023-12-08,1383414,None,None,0,https://www.sec.gov/Archives/edgar/data/138341...,0000950170-23-068909


In [13]:
results_df = pl.from_pandas(results_df)

In [16]:
results_df

company_name,filing_date,cik,trade_secrets,net_trade_secrets,indicator,url,accession_no
str,date,i64,f64,str,i64,str,str
"""ACCENTIA BIOPH…",2006-12-29,1310094,null,null,1,"""https://www.se…","""0001193125-06-…"
"""ACTIVIDENTITY …",2006-12-29,1183941,null,null,1,"""https://www.se…","""0001193125-06-…"
"""ALLSTATES WORL…",2006-12-29,1072293,null,null,1,"""https://www.se…","""0001260415-06-…"
"""ALTIGEN COMMUN…",2006-12-29,1003607,null,null,1,"""https://www.se…","""0001104659-06-…"
"""AMCON DISTRIBU…",2006-12-29,928465,null,null,0,"""https://www.se…","""0000928465-06-…"
"""APPLE COMPUTER…",2006-12-29,320193,null,null,1,"""https://www.se…","""0001104659-06-…"
"""ARK RESTAURANT…",2006-12-29,779544,null,null,0,"""https://www.se…","""0000930413-06-…"
"""BIOANALYTICAL …",2006-12-29,720154,null,null,0,null,"""0000927946-06-…"
"""Brooklyn Feder…",2006-12-29,1310313,null,null,0,"""https://www.se…","""0001188112-06-…"


In [17]:
urls = results_df['url'].to_list()

In [18]:
urls

['https://www.sec.gov/Archives/edgar/data/1310094/000119312506261727/d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1183941/000119312506261251/d10k.htm',
 'https://www.sec.gov/Archives/edgar/data/1072293/000126041506000061/awc10k09-06.txt',
 'https://www.sec.gov/Archives/edgar/data/1003607/000110465906084411/a06-26403_310k.htm',
 'https://www.sec.gov/Archives/edgar/data/928465/000092846506000077/sept200610k.txt',
 'https://www.sec.gov/Archives/edgar/data/320193/000110465906084288/a06-25759_210k.htm',
 'https://www.sec.gov/Archives/edgar/data/779544/000093041306008983/c45950_10-k.txt',
 None,
 'https://www.sec.gov/Archives/edgar/data/1310313/000118811206003824/t12043_10k.txt',
 'https://www.sec.gov/Archives/edgar/data/1063085/000089161806000545/f24355e10vk.htm',
 'https://www.sec.gov/Archives/edgar/data/874292/000135585607000062/form10k.htm',
 'https://www.sec.gov/Archives/edgar/data/860543/000114554907002328/h01759e10vk.htm',
 'https://www.sec.gov/Archives/edgar/data/1310313/0001

In [23]:
filing_10k_by_year[2006]

╭──────────────────────────────────────────────────── Filings ────────────────────────────────────────────────────╮
│                                                                                                                 │
│       form   company                           cik       filing_date   accession_number                         │
│  ───────────────────────────────────────────────────────────────────────────────────────────                    │
│   0   10-K   ACCENTIA BIOPHARMACEUTICALS INC   1310094   2006-12-29    0001193125-06-261727                     │
│   1   10-K   ACTIVIDENTITY CORP                1183941   2006-12-29    0001193125-06-261251                     │
│   2   10-K   ALLSTATES WORLDCARGO INC          1072293   2006-12-29    0001260415-06-000061                     │
│   3   10-K   ALTIGEN COMMUNICATIONS INC        1003607   2006-12-29    0001104659-06-084411                     │
│   4   10-K   AMCON DISTRIBUTING CO             928465    2006-12-29   

In [26]:
filing_10k_by_year[2006][7]

[22:58:34] INFO     HTTP Request: GET                                                               ]8;id=441157;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=819201;file:///home/jason/anaconda3/lib/python3.10/site-packages/httpx/_client.py#1013\1013]8;;\
                    https://www.sec.gov/Archives/edgar/data/720154/0000927946-06-000172-index.html                 
                    "HTTP/1.1 200 OK"                                                                              

╭───────────────────────────────── 10-K 📊 filing for BIOANALYTICAL SYSTEMS INC ──────────────────────────────────╮
│                                                                                                                 │
│   Accession Number       Filing Date   Company                     CIK                                          │
│  ─────────────────────────────────────────────────────────────────────────                                      │
│   0000927946-06-000172   2006-12-29    BIOANALYTICAL SYSTEMS INC   720154                                       │
│                                                                                                                 │
│                                                                                                                 │
│   Link                      URL                                                                                 │
│  ──────────────────────────────────────────────────────────────────────

In [21]:
filing_10k[2006]

IndexError: index out of bounds